# JSONB runbook

In [ ]:
%%bash
docker run --name jsonb -d -e POSTGRES_HOST_AUTH_METHOD=trust -p 5432:5432 postgres:16.2
sleep 2

In [ ]:
%%bash
psql -h localhost -U postgres -d postgres -c "

CREATE TABLE IF NOT EXISTS document (
    id BIGSERIAL PRIMARY KEY,
    data JSONB
);

"

In [ ]:
import psycopg
from psycopg.abc import Query

def insert(data: str) -> tuple[str, str] | None:
    with psycopg.connect("host=localhost dbname=postgres user=postgres") as conn:
        with conn.cursor() as cur:
            cur.execute("""
                INSERT INTO document (data)
                VALUES (%s)
                RETURNING *
                """, [data])
            return cur.fetchone()


def query(sql: Query) -> list[tuple[str, str]] | None:
    with psycopg.connect("host=localhost dbname=postgres user=postgres") as conn:
        with conn.cursor() as cur:
            cur.execute(sql)
            return cur.fetchall()

In [ ]:
import json

vemund = {
    'name': 'vemund'
}

kristian = {
    'name': 'kristian',
    'favourite_food': 'taco',
    'pets': [
        {
            'name': 'yava',
            'species': 'dog',
            'age': 5
        },
        {
            'name': 'shell',
            'species': 'fish',
            'age': 2
        },
        {
            'name': 'rust',
            'species': 'cat',
            'age': 7
        },
    ]
}

trine = {
    'name': 'trine',
    'food_allergies': ['molluscs', 'soy'],
    'pets': [
        {
            'name': 'barten',
            'species': 'cat',
            'age': 7
        },
    ]
}

insert(json.dumps(vemund))
insert(json.dumps(kristian))
insert(json.dumps(trine))


In [ ]:
# Get Vemund
query("""
      SELECT *
      FROM document
      WHERE data ->> 'name' = 'vemund'
""")

In [ ]:
# Get all documents that has a fish as a pet
query("""
    SELECT *
    FROM document
    WHERE data -> 'pets' @> '[{"species":"fish"}]'::jsonb
""")

In [ ]:
# Select Vemund and create a list of his friends as a property on Vemund
query("""
    SELECT jsonb_set(
        data,
        '{friends}',
        (
            SELECT jsonb_agg(data)
            FROM document
            WHERE data ->> 'name' IN ('kristian', 'trine')
        )
    )
    FROM document
    WHERE data ->> 'name' = 'vemund'
""")



In [ ]:
%%bash
psql -h localhost -U postgres -d postgres -c "

CREATE TABLE IF NOT EXISTS employee (
    name TEXT PRIMARY KEY,
    company TEXT NOT NULL
);
"

In [ ]:
%%bash
psql -h localhost -U postgres -d postgres -c "

INSERT INTO employee (name, company)
VALUES ('kristian', 'Oiiku'),
       ('trine', 'Age Labs');
"

In [ ]:
# Select Vemund and create a list of his friends as a property on Vemund, then add the company they work for
query("""
    SELECT jsonb_set(
        data,
        '{friends}',
        (
            SELECT jsonb_agg(jsonb_set(
                data,
                '{company}',
                to_jsonb((SELECT company FROM employee WHERE name = data ->> 'name'))
            ))
            FROM document
            WHERE data ->> 'name' IN ('kristian', 'trine')
        )
    )
    FROM document
    WHERE data ->> 'name' = 'vemund'
""")

In [ ]:
%%bash
docker stop jsonb
docker rm jsonb